https://github.com/vanvalenlab/deepcell-tf/blob/master/scripts/misc/Model_Metrics.ipynb

In [1]:
import os
import sys

sys.path.insert(0,'/home/sunnycui/deepcell-tf/') 

import numpy as np

import matplotlib.pyplot as plt
from skimage.io import imshow

from skimage.measure import label
from skimage import morphology

import random
np.random.seed(2019)
random.seed(2019)

from deepcell import metrics

MODEL_DIR = os.path.join(sys.path[0], 'scripts/recurr_gru/models')

Instructions for updating:
Colocations handled automatically by placer.


In [2]:
from deepcell.utils.data_utils import get_data
from tensorflow.python.keras.utils.data_utils import get_file

data_filename = 'combined_data.npz'
# data_filename = '3T3_NIH.npz'
data_path = os.path.join(sys.path[0], 'scripts/recurr_gru/data', data_filename)

if not os.path.isfile(data_path):
    print('downloading ' + data_filename)
    data_path = get_file(data_filename,
                    origin='https://deepcell-data.s3.amazonaws.com/nuclei/3T3_NIH.npz',
                    file_hash='954b6f4ad6a71435b84c40726837e4ba')

print("Loading data from " + data_filename)
train_dict, test_dict = get_data(data_path, test_size=0.1, seed=0)
print('X.shape: {}\ny.shape: {}'.format(test_dict['X'].shape, test_dict['y'].shape))

Loading data from combined_data.npz
X.shape: (68, 30, 135, 160, 1)
y.shape: (68, 30, 135, 160, 1)


In [3]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import callbacks
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.layers import MaxPool3D, Conv3DTranspose, UpSampling3D
from scripts.recurr_gru.conv_gru_layer import ConvGRU2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout, LeakyReLU
from tensorflow.python.keras.layers import Conv3D, ZeroPadding3D, ConvLSTM2D, Cropping3D
from tensorflow.python.keras.layers import Input, Add, Concatenate, Flatten
from tensorflow.python.keras.engine.input_layer import InputLayer

from tensorflow.python.keras.models import Model

from tensorflow.python.keras.regularizers import l2
from deepcell.layers import ImageNormalization3D
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Softmax

from deepcell.layers import TensorProduct, ReflectionPadding3D, DilatedMaxPool3D


In [4]:
def feature_net_3D(receptive_field=61,
                    n_frames=3,
                    input_shape=(5, 256, 256, 1),
                    n_features=3,
                    n_channels=1,
                    reg=1e-5,
                    n_conv_filters=64,
                    n_dense_filters=200,
                    gru_kernel_size =3,
                    VGG_mode=False,
                    init='he_normal',
                    norm_method='whole_image',
                    gru=False,
                    location=False,
                    dilated=False,
                    padding=False,
                    padding_mode='reflect',
                    multires=False,
                    include_top=True):
    # Create layers list (x) to store all of the layers.
    # We need to use the functional API to enable the multiresolution mode
    x = []

    win = (receptive_field - 1) // 2
    win_z = (n_frames - 1) // 2

    if dilated:
        padding = True

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
        time_axis = 2
        row_axis = 3
        col_axis = 4
        if not dilated:
            input_shape = (n_channels, n_frames, receptive_field, receptive_field)
    else:
        channel_axis = -1
        time_axis = 1
        row_axis = 2
        col_axis = 3
        if not dilated:
            input_shape = (n_frames, receptive_field, receptive_field, n_channels)

    x.append(Input(shape=input_shape))
    # x.append(ImageNormalization3D(norm_method=norm_method, filter_size=receptive_field)(x[-1]))
    # x.append(BatchNormalization(axis=channel_axis)(x[-1]))

    if padding:
        if padding_mode == 'reflect':
            x.append(ReflectionPadding3D(padding=(win_z, win, win))(x[-1]))
        elif padding_mode == 'zero':
            x.append(ZeroPadding3D(padding=(win_z, win, win))([-1]))

    if location:
        x.append(Location3D(in_shape=tuple(x[-1].shape.as_list()[1:]))(x[-1]))
        x.append(Concatenate(axis=channel_axis)([x[-2], x[-1]]))

    if multires:
        layers_to_concat = []

    rf_counter = receptive_field
    block_counter = 0
    d = 1

    append_gru = False
    while rf_counter > 4:
        filter_size = 3 if rf_counter % 2 == 0 else 4
        if append_gru == True:
            x.append(ConvGRU2D(filters=n_conv_filters, kernel_size=(filter_size, filter_size), dilation_rate=(d, d),
                            padding='valid', kernel_initializer=init,
                            kernel_regularizer=l2(reg), return_sequences=True)(x[-1]))
        else:
            x.append(Conv3D(n_conv_filters, (1, filter_size, filter_size), 
                            dilation_rate=(1, d, d), kernel_initializer=init,
                            padding='valid', kernel_regularizer=l2(reg))(x[-1]))

        
        x.append(BatchNormalization(axis=channel_axis)(x[-1]))
        x.append(Activation('relu')(x[-1]))

        block_counter += 1
        rf_counter -= filter_size - 1

        if block_counter % 2 == 0:
            if dilated:
                x.append(DilatedMaxPool3D(dilation_rate=(1, d, d), pool_size=(1, 2, 2))(x[-1]))
                d *= 2
            else:
                x.append(MaxPool3D(pool_size=(1, 2, 2))(x[-1]))

            if VGG_mode:
                n_conv_filters *= 2

            rf_counter = rf_counter // 2

            if multires:
                layers_to_concat.append(len(x) - 1)

    if multires:
        c = []
        for l in layers_to_concat:
            output_shape = x[l].get_shape().as_list()
            target_shape = x[-1].get_shape().as_list()
            time_crop = (0, 0)

            row_crop = int(output_shape[row_axis] - target_shape[row_axis])

            if row_crop % 2 == 0:
                row_crop = (row_crop // 2, row_crop // 2)
            else:
                row_crop = (row_crop // 2, row_crop // 2 + 1)

            col_crop = int(output_shape[col_axis] - target_shape[col_axis])

            if col_crop % 2 == 0:
                col_crop = (col_crop // 2, col_crop // 2)
            else:
                col_crop = (col_crop // 2, col_crop // 2 + 1)

            cropping = (time_crop, row_crop, col_crop)

            c.append(Cropping3D(cropping=cropping)(x[l]))
        x.append(Concatenate(axis=channel_axis)(c))
        
    
    x.append(Conv3D(n_dense_filters, (1, rf_counter, rf_counter), dilation_rate=(1, d, d), kernel_initializer=init, padding='valid', kernel_regularizer=l2(reg))(x[-1]))
    x.append(BatchNormalization(axis=channel_axis)(x[-1]))
    x.append(Activation('relu')(x[-1]))

    x.append(Conv3D(n_dense_filters, (n_frames, 1, 1), dilation_rate=(1, d, d), kernel_initializer=init, padding='valid', kernel_regularizer=l2(reg))(x[-1]))
    x.append(BatchNormalization(axis=channel_axis)(x[-1]))
    x.append(Activation('relu')(x[-1]))
    
    if gru == True:
        x.append(ConvGRU2D(filters=n_conv_filters, kernel_size=(gru_kernel_size, gru_kernel_size),
                            padding='same', kernel_initializer=init, activation='relu',
                            kernel_regularizer=l2(reg), return_sequences=True)(x[-1]))
        x.append(BatchNormalization(axis=channel_axis)(x[-1]))
        # x.append(Activation('relu')(x[-1]))
        x.append(ConvGRU2D(filters=n_conv_filters, kernel_size=(gru_kernel_size +2, gru_kernel_size +2),
                            padding='same', kernel_initializer=init, activation='relu',
                            kernel_regularizer=l2(reg), return_sequences=True)(x[-1]))
        x.append(BatchNormalization(axis=channel_axis)(x[-1]))
        # x.append(Activation('relu')(x[-1]))
    
    
    x.append(TensorProduct(n_dense_filters, kernel_initializer=init, kernel_regularizer=l2(reg))(x[-1]))
    x.append(BatchNormalization(axis=channel_axis)(x[-1]))
    x.append(Activation('relu')(x[-1]))

    x.append(TensorProduct(n_features, kernel_initializer=init, kernel_regularizer=l2(reg))(x[-1]))

    if not dilated:
        x.append(Flatten()(x[-1]))

    if include_top:
        x.append(Softmax(axis=channel_axis)(x[-1]))

    model = Model(inputs=x[0], outputs=x[-1])
    # model.summary()

    return model

In [5]:
from tensorflow.python.keras.layers import Lambda;

def image_norm(inputs):
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1
    axes = [3, 4] if channel_axis == 1 else [2, 3]
    output = inputs - K.mean(inputs, axis=axes, keepdims=True)
    output = output / K.std(inputs, axis=axes, keepdims=True)
    return output

def feature_net_skip_3D(receptive_field=61,
                        input_shape=(5, 256, 256, 1),
                        fgbg_model=None,
                        gru=False,
                        last_only=True,
                        n_skips=1,
                        norm_method='whole_image',
                        padding_mode='reflect',
                        **kwargs):
    # print(K.image_data_format())
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1

    inputs = Input(shape=input_shape)
    
    img = Lambda(image_norm)(inputs)

    models = []
    model_outputs = []

    if fgbg_model is not None:
        for layer in fgbg_model.layers:
            layer.trainable = False
        models.append(fgbg_model)
        fgbg_output = fgbg_model(inputs)
        if isinstance(fgbg_output, list):
            fgbg_output = fgbg_output[-1]
        model_outputs.append(fgbg_output)

    for _ in range(n_skips + 1):
        if model_outputs:
            model_input = Concatenate(axis=channel_axis)([img, model_outputs[-1]])
        else:
            model_input = img
        new_input_shape = model_input.get_shape().as_list()[1:]
        models.append(feature_net_3D(receptive_field=receptive_field, 
                                     input_shape=new_input_shape, norm_method=None, dilated=True, 
                                     padding=True, padding_mode=padding_mode, gru=gru, **kwargs))
        model_outputs.append(models[-1](model_input))

    if last_only:
        model = Model(inputs=inputs, outputs=model_outputs[-1])
    else:
        if fgbg_model is None:
            model = Model(inputs=inputs, outputs=model_outputs)
        else:
            model = Model(inputs=inputs, outputs=model_outputs[1:])

    return model

In [6]:
def get_gru_model(gru_kernel_size=3):
    fgbg_gru_model = feature_net_skip_3D(
                    receptive_field=61,
                    n_features=2,
                    n_frames=3,
                    n_skips=1,
                    gru=True,
                    gru_kernel_size=gru_kernel_size,
                    n_conv_filters=32,
                    n_dense_filters=128,
                    input_shape=tuple(train_dict['X'].shape[1:]),
                    multires=False,
                    last_only=False,
                    norm_method='wholeimage')

    conv_gru_model = feature_net_skip_3D(
                    # fgbg_model=run_fgbg_model,
                    receptive_field=61,
                    n_skips=1,
                    n_features=4,  # (background edge, interior edge, cell interior, background)
                    n_frames=3,
                    n_conv_filters=32,
                    n_dense_filters=128,
                    gru=True,
                    gru_kernel_size=gru_kernel_size,
                    multires=False,
                    last_only=False,
                    input_shape=tuple(train_dict['X'].shape[1:]),
                    norm_method='whole_image')
    return fgbg_gru_model, conv_gru_model

In [7]:
def get_baseline_model():
    fgbg_model = feature_net_skip_3D(
            receptive_field=61,
            n_features=2,
            n_frames=3,
            n_skips=1,
            n_conv_filters=32,
            n_dense_filters=128,
            gru=False,
            input_shape=tuple(train_dict['X'].shape[1:]),
            multires=False,
            last_only=False,
            norm_method='whole_image')

    conv_model = feature_net_skip_3D(
            # fgbg_model=run_fgbg_model,
            receptive_field=61,
            n_skips=1,
            n_features=4,  # (background edge, interior edge, cell interior, background)
            n_frames=3,
            n_conv_filters=32,
            n_dense_filters=128,
            gru=False,
            multires=False,
            last_only=False,
            input_shape=tuple(train_dict['X'].shape[1:]),
            norm_method='whole_image')
    return fgbg_model, conv_model

# Post Processing

In [8]:
def post_process(test_images, test_images_fgbg):
    argmax_images = []
    for i in range(test_images.shape[0]):
        max_image = np.argmax(test_images[i], axis=-1)
        argmax_images.append(max_image)
    argmax_images = np.array(argmax_images)
    argmax_images = np.expand_dims(argmax_images, axis=-1)

    print('argmax shape:', argmax_images.shape)

    # threshold the foreground/background
    # and remove back ground from edge transform
    threshold = 0.9

    fg_thresh = test_images_fgbg[..., 1] > threshold
    fg_thresh = np.expand_dims(fg_thresh, axis=-1)

    test_images_post_fgbg = test_images * fg_thresh


    # Label interior predictions

    labeled_images = []
    for i in range(test_images_post_fgbg.shape[0]):
        interior = test_images_post_fgbg[i, ..., 2] > .2
        labeled_image = label(interior)
        labeled_image = morphology.remove_small_objects(
            labeled_image, min_size=50, connectivity=1)
        labeled_images.append(labeled_image)
    labeled_images = np.array(labeled_images)
    labeled_images = np.expand_dims(labeled_images, axis=-1)

    print('labeled_images shape:', labeled_images.shape)

    return labeled_images, fg_thresh

In [9]:
test_size = .10  # % of data saved as test
receptive_field = 61  # should be adjusted for the scale of the data
optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# FC training settings
n_skips = 1 # number of skip-connections (only for FC training)
batch_size = 1  # FC training uses 1 image per batch

# Transformation settings
transform = 'deepcell_flat' # 'watershed'
dilation_radius = 1  # change dilation radius for edge dilation
n_features = 4  # (cDell-background edge, cell-cell edge, cell interior, background)

# 3D Settings
frames_per_batch = 3
norm_method = None #'whole_image'


def test_gru(X_test, fgbg_gru_weights_file, conv_gru_weights_file, gru_kernel_size=3, gru=False):
    run_fgbg_model = feature_net_skip_3D(
        receptive_field=receptive_field,
        n_features=2,
        n_frames=frames_per_batch,
        n_skips=n_skips,
        n_conv_filters=32,
        n_dense_filters=128,
        input_shape=tuple(X_test.shape[1:]),
        gru=gru,
        gru_kernel_size=gru_kernel_size,
        multires=False,
        last_only=False,
        norm_method=norm_method)

    run_fgbg_model.load_weights(fgbg_gru_weights_file)

    
    run_conv_model = feature_net_skip_3D(
        receptive_field=receptive_field,
        n_skips=n_skips,
        n_features=4,  # (background edge, interior edge, cell interior, background)
        n_frames=frames_per_batch,
        n_conv_filters=32,
        n_dense_filters=128,
        gru=gru,
        gru_kernel_size=gru_kernel_size,
        multires=False,
        last_only=False,
        input_shape=tuple(X_test.shape[1:]),
        norm_method=norm_method)
    run_conv_model.load_weights(conv_gru_weights_file)
    
    test_images = run_conv_model.predict(X_test)[-1]
    test_images_fgbg = run_fgbg_model.predict(X_test)[-1]

    print('edge/interior prediction shape:', test_images.shape)
    print('fgbg mask shape:', test_images_fgbg.shape)

    return test_images, test_images_fgbg

# Metrics functions

In [10]:
from deepcell import metrics
from statistics import mean 
import decimal


def to_precision(x, p):
    """
    returns a string representation of x formatted with a precision of p

    Based on the webkit javascript implementation taken from here:
    https://code.google.com/p/webkit-mirror/source/browse/JavaScriptCore/kjs/number_object.cpp
    """

    decimal.getcontext().prec = p
    return decimal.Decimal(x)

def get_metrics(model, test_dict_X, test_dict_y, fgbg_weights, conv_weights, gru):
    if model == 'fgbg':
        m = metrics.Metrics('fgbg',seg=False)
    elif model == 'segmentation':
        m = metrics.Metrics('deepcell',seg=True)

    n_true = []
    n_pred = []
    correct_detections = []
    gained_detections = []
    missed_detections = []
    split = []
    merge = []
    catastrophe = []
    seg = []
    jaccard = []
    gained_det_from_split = []
    missed_det_from_merge = []
    true_det_in_catastrophe = []
    pred_det_in_catastrophe = []

    for i in range(0, len(test_dict_y), 6):
        top = min(i+6, len(test_dict_y))
        X_test, y_test = test_dict_X[i:top], test_dict_y[i:top]
        test_images, test_images_fgbg = test_gru(X_test, fgbg_weights, conv_weights, 
                                             gru_kernel_size=3, gru=gru)

        y_pred, fg_thresh = post_process(test_images, test_images_fgbg)
        if y_test.shape != y_pred.shape:
            print("y_test.shape != y_pred.shape")
            print("y_test.shape: ", y_test.shape)
            print("y_pred.shape: ", y_pred.shape)
        
        y_true_lbl = y_test[:,:,:,0].astype('int')
        y_pred_lbl = y_pred[:,:,:,0]
                
        m.calc_object_stats(y_true_lbl,y_pred_lbl)
        
        n_true.append(m.stats['n_true'].sum())
        n_pred.append(m.stats['n_pred'].sum())
        correct_detections.append(m.stats['correct_detections'].sum())
        missed_detections.append(m.stats['missed_detections'].sum())
        gained_detections.append(m.stats['gained_detections'].sum())
        catastrophe.append(m.stats['catastrophe'].sum())
        merge.append(m.stats['merge'].sum())
        split.append(m.stats['split'].sum())
        
        gained_det_from_split.append(m.stats['gained_det_from_split'].sum())
        missed_det_from_merge.append(m.stats['missed_det_from_merge'].sum())
        true_det_in_catastrophe.append(m.stats['true_det_in_catastrophe'].sum())
        pred_det_in_catastrophe.append(m.stats['pred_det_in_catastrophe'].sum())
        
        jaccard.extend(m.stats['jaccard'])
        
        if m.seg is True:
            seg = seg + m.stats['seg']
        
    print('\n____________Object-based statistics____________\n')
    print('Number of true cells:\t\t', sum(n_true))
    print('Number of predicted cells:\t', sum(n_pred))

    print('\nCorrect detections:  {}\tRecall: {}%'.format(
        int(sum(correct_detections)),
        to_precision(100 * sum(correct_detections) / sum(n_true),
                     m.ndigits)))
    print('Incorrect detections: {}\tPrecision: {}%'.format(
        int(sum(n_pred) - sum(correct_detections)),
        to_precision(100 * sum(correct_detections) / sum(n_pred),
                     m.ndigits)))

    total_err = (sum(gained_detections)
                 + sum(missed_detections)
                 + sum(split)
                 + sum(merge)
                 + sum(catastrophe))

    print('\nGained detections: {}\tPerc Error: {}%'.format(
        int(sum(gained_detections)),
        to_precision(100 * sum(gained_detections) / total_err, m.ndigits)))
    print('Missed detections: {}\tPerc Error: {}%'.format(
        int(sum(missed_detections)),
        to_precision(100 * sum(missed_detections) / total_err, m.ndigits)))
    print('Merges: {}\t\tPerc Error: {}%'.format(
        int(sum(merge)),
        to_precision(100 * sum(merge) / total_err, m.ndigits)))
    print('Splits: {}\t\tPerc Error: {}%'.format(
        int(sum(split)),
        to_precision(100 * sum(split) / total_err, m.ndigits)))
    print('Catastrophes: {}\t\tPerc Error: {}%\n'.format(
        int(sum(catastrophe)),
        to_precision(100 * sum(catastrophe) / total_err, m.ndigits)))

    print('Gained detections from splits: {}'.format(
        int(sum(gained_det_from_split))))
    print('Missed detections from merges: {}'.format(
        int(sum(missed_det_from_merge))))
    print('True detections involved in catastrophes: {}'.format(
        int(sum(true_det_in_catastrophe))))
    print('Predicted detections involved in catastrophes: {}'.format(
        int(sum(pred_det_in_catastrophe))), '\n')

    if m.seg is True:
        print('SEG:', to_precision(mean(seg), m.ndigits), '\n')

    print('Average Pixel IOU (Jaccard Index):',
          to_precision(mean(jaccard), m.ndigits), '\n')

#     return n_true, n_pred, correct_detections, gained_detections, 
#         missed_detections, split, merge, catastrophe, seg, jaccard, 
#         gained_det_from_split, missed_det_from_merge, 
#         true_det_in_catastrophe, pred_det_in_catastrophe

# Feature net 3D (Baseline)

In [11]:
# Load weights
fgbg_model_name = 'fgbg_featurenet_model_full'
fgbg_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(fgbg_model_name))

conv_model_name = 'conv_featurenet_model_full'
conv_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(conv_model_name))

get_metrics('fgbg', test_dict['X'], test_dict['y'], fgbg_weights_file, conv_weights_file, False)

edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
argmax shape: (6, 30, 135, 160, 1)
labeled_images shape: (6, 30, 135, 160, 1)
INFO:tensorflow:Ground truth frame is empty
INFO:tensorflow:0 samples processed


/home/sunnycui/deepcell-tf/deepcell/metrics.py:113: RuntimeWarning: invalid value encountered in long_scalars
  dice = (2 * intersection.sum() / (pred.sum() + truth.sum()))
/home/sunnycui/deepcell-tf/deepcell/metrics.py:114: RuntimeWarning: invalid value encountered in long_scalars
  jaccard = intersection.sum() / union.sum()
/home/sunnycui/deepcell-tf/deepcell/metrics.py:115: RuntimeWarning: invalid value encountered in long_scalars
  precision = intersection.sum() / pred.sum()
/home/sunnycui/deepcell-tf/deepcell/metrics.py:116: RuntimeWarning: invalid value encountered in long_scalars
  recall = intersection.sum() / truth.sum()


edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
argmax shape: (6, 30, 135, 160, 1)
labeled_images shape: (6, 30, 135, 160, 1)
INFO:tensorflow:0 samples processed
edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
argmax shape: (6, 30, 135, 160, 1)
labeled_images shape: (6, 30, 135, 160, 1)
INFO:tensorflow:0 samples processed
edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
argmax shape: (6, 30, 135, 160, 1)
labeled_images shape: (6, 30, 135, 160, 1)
INFO:tensorflow:0 samples processed
INFO:tensorflow:Ground truth frame is empty
edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
argmax shape: (6, 30, 135, 160, 1)
labeled_images shape: (6, 30, 135, 160, 1)
INFO:tensorflow:0 samples processed
INFO:tensorflow:Ground truth frame is empty
edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
a

### Pixelwise

In [13]:
counter = 0
pixel_df = None
cms = []
import pandas as pd 

fgbg_model_name = 'fgbg_featurenet_model_full'
fgbg_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(fgbg_model_name))

conv_model_name = 'conv_featurenet_model_full'
conv_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(conv_model_name))
m = metrics.Metrics('fgbg',seg=False)

for i in range(0, len(test_dict['y']), 6):
    top = min(i+6, len(test_dict['y']))
    X_test, y_test = test_dict['X'][i:top], test_dict['y'][i:top]
    test_images, test_images_fgbg = test_gru(X_test, fgbg_weights_file, conv_weights_file,
                                     gru_kernel_size=3, gru=False)
    y_pred, fg_thresh = post_process(test_images, test_images_fgbg)
    m.all_pixel_stats(y_test, y_pred)
    cms.append(m.cm)
    if counter == 0:
        pixel_df = m.pixel_df
    else:
        pixel_df = pixel_df.append(m.pixel_df)

edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
argmax shape: (6, 30, 135, 160, 1)
labeled_images shape: (6, 30, 135, 160, 1)

____________Pixel-based statistics____________

       dice   jaccard  precision    recall  Fmeasure
0  0.683667  0.519372   0.927727  0.541273  0.683667

Confusion Matrix
[[3888000]]
edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
argmax shape: (6, 30, 135, 160, 1)
labeled_images shape: (6, 30, 135, 160, 1)

____________Pixel-based statistics____________

       dice   jaccard  precision    recall  Fmeasure
0  0.793892  0.658227   0.966374  0.673656  0.793892

Confusion Matrix
[[3888000]]
edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
argmax shape: (6, 30, 135, 160, 1)
labeled_images shape: (6, 30, 135, 160, 1)

____________Pixel-based statistics____________

       dice   jaccard  precision    recall  Fmeasure
0  0.795701  0.660717

In [30]:
import pandas as pd 

print('\n____________Pixel-based statistics____________\n')
temp_df = pd.DataFrame([[0.683667,  0.519372,   0.927727,  0.541273,  0.683667],
[0.793892,  0.658227,   0.966374,  0.673656,  0.793892],
[0.795701,  0.660717,   0.995482,  0.662704,  0.795701],
[0.755852,  0.607526,   0.992752,  0.610233,  0.755852],
[0.618439,  0.447638,   0.592187,  0.647126,  0.618439],
[0.802582,  0.670261,   0.896204,  0.726671,  0.802582],
[0.801129,  0.668237,   0.961278,  0.686722,  0.801129],
[0.820884,  0.696186,   0.941316,  0.727773,  0.820884],
[0.712351,  0.553219,   0.946152,  0.571203,  0.712351],
[0.774828,  0.632424,     0.9738,  0.643372,  0.774828],
[0.754551,  0.605847,   0.756009,  0.7531,  0.754551],
[0.789015,  0.651547,   0.985472,  0.657866,  0.789015]], 
                       columns = ["dice",   "jaccard",  "precision",    "recall",  "Fmeasure"])
print(temp_df.mean())
print('\nConfusion Matrix')
temp_cms = [x[0][0] for x in cms[:-1]]
print(mean(temp_cms))


____________Pixel-based statistics____________

dice         0.758574
jaccard      0.614267
precision    0.911229
recall       0.658475
Fmeasure     0.758574
dtype: float64

Confusion Matrix
3888000


# Feature net with ConvGRU

###  Kernel size (3, 3) and (5, 5)

In [11]:
# Load weights
fgbg_gru_model_name = 'fgbg_gru_featurenet_model_full'
fgbg_gru_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(fgbg_gru_model_name))

conv_gru_model_name = 'conv_gru_featurenet_model_full'
conv_gru_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(conv_gru_model_name))

# Predict
get_metrics('fgbg', test_dict['X'], test_dict['y'], fgbg_gru_weights_file, conv_gru_weights_file, gru=True)

edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
argmax shape: (6, 30, 135, 160, 1)
labeled_images shape: (6, 30, 135, 160, 1)
INFO:tensorflow:Ground truth frame is empty
INFO:tensorflow:0 samples processed


/home/sunnycui/deepcell-tf/deepcell/metrics.py:113: RuntimeWarning: invalid value encountered in long_scalars
  dice = (2 * intersection.sum() / (pred.sum() + truth.sum()))
/home/sunnycui/deepcell-tf/deepcell/metrics.py:114: RuntimeWarning: invalid value encountered in long_scalars
  jaccard = intersection.sum() / union.sum()
/home/sunnycui/deepcell-tf/deepcell/metrics.py:115: RuntimeWarning: invalid value encountered in long_scalars
  precision = intersection.sum() / pred.sum()
/home/sunnycui/deepcell-tf/deepcell/metrics.py:116: RuntimeWarning: invalid value encountered in long_scalars
  recall = intersection.sum() / truth.sum()
/home/sunnycui/deepcell-tf/deepcell/metrics.py:117: RuntimeWarning: invalid value encountered in double_scalars
  Fmeasure = (2 * precision * recall) / (precision + recall)


edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
argmax shape: (6, 30, 135, 160, 1)
labeled_images shape: (6, 30, 135, 160, 1)
INFO:tensorflow:0 samples processed
edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
argmax shape: (6, 30, 135, 160, 1)
labeled_images shape: (6, 30, 135, 160, 1)
INFO:tensorflow:0 samples processed
INFO:tensorflow:Prediction frame is empty
edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
argmax shape: (6, 30, 135, 160, 1)
labeled_images shape: (6, 30, 135, 160, 1)
INFO:tensorflow:0 samples processed
INFO:tensorflow:Ground truth frame is empty
edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
argmax shape: (6, 30, 135, 160, 1)
labeled_images shape: (6, 30, 135, 160, 1)
INFO:tensorflow:0 samples processed
INFO:tensorflow:Ground truth frame is empty
edge/interior prediction shape: (6, 30, 135, 160, 

### Pixelwise

In [17]:
counter = 0
pixel_df = None
cms = []

fgbg_gru_model_name = 'fgbg_gru_featurenet_model_full'
fgbg_gru_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(fgbg_gru_model_name))

conv_gru_model_name = 'conv_gru_featurenet_model_full'
conv_gru_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(conv_gru_model_name))
m = metrics.Metrics('fgbg',seg=False)

for i in range(0, len(test_dict['y']), 6):
    top = min(i+6, len(test_dict['y']))
    X_test, y_test = test_dict['X'][i:top], test_dict['y'][i:top]
    test_images, test_images_fgbg = test_gru(X_test, fgbg_gru_weights_file, conv_gru_weights_file,
                                     gru_kernel_size=3, gru=True)
    y_pred, fg_thresh = post_process(test_images, test_images_fgbg)
    m.all_pixel_stats(y_test, y_pred)
    cms.append(m.cm)
    if counter == 0:
        pixel_df = m.pixel_df
    else:
        pixel_df = pixel_df.append(m.pixel_df)

edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
argmax shape: (6, 30, 135, 160, 1)
labeled_images shape: (6, 30, 135, 160, 1)

____________Pixel-based statistics____________

       dice   jaccard  precision    recall  Fmeasure
0  0.343607  0.207443   0.928009  0.210836  0.343607

Confusion Matrix
[[3888000]]
edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
argmax shape: (6, 30, 135, 160, 1)
labeled_images shape: (6, 30, 135, 160, 1)

____________Pixel-based statistics____________

      dice  jaccard  precision    recall  Fmeasure
0  0.58157  0.41001   0.962494  0.416667   0.58157

Confusion Matrix
[[3888000]]
edge/interior prediction shape: (6, 30, 135, 160, 4)
fgbg mask shape: (6, 30, 135, 160, 2)
argmax shape: (6, 30, 135, 160, 1)
labeled_images shape: (6, 30, 135, 160, 1)

____________Pixel-based statistics____________

       dice   jaccard  precision    recall  Fmeasure
0  0.653242  0.485048   0

In [32]:
print('\n____________Pixel-based statistics____________\n')
temp_df = pd.DataFrame([[0.343607,  0.207443,   0.928009,  0.210836,  0.343607],
[0.58157,  0.41001,   0.962494,  0.416667,   0.58157],
[0.653242,  0.485048,   0.997939,  0.485534,  0.653242],
[0.533109,  0.363428,   0.996458,  0.363898,  0.533109],
[0.504673,   0.3375,   0.557467,  0.461013,  0.504673],
[0.46404,  0.302117,   0.935065,  0.308591,   0.46404],
[0.54703,  0.376491,   0.962544,  0.382089,   0.54703],
[0.623734,  0.453208,   0.929236,  0.469409,  0.623734],
[0.418123,  0.264321,   0.916296,  0.270861,  0.418123],
[0.579613,  0.408067,   0.982024,  0.411139,  0.579613],
[0.528312,  0.358984,    0.73029,  0.413852,  0.528312],
[0.408233,  0.256466,        1.0,  0.256466,  0.408233]], 
                       columns = ["dice",   "jaccard",  "precision",    "recall",  "Fmeasure"])
print(temp_df.mean())
print('\nConfusion Matrix')
temp_cm = [3888000, 3888000, 3888000, 3888000, 3888000, 3888000, 
3888000, 3888000, 3888000, 3888000, 3888000, 1296000]
print(mean(temp_cm[:-1]))


____________Pixel-based statistics____________

dice         0.515441
jaccard      0.351924
precision    0.908152
recall       0.370863
Fmeasure     0.515441
dtype: float64

Confusion Matrix
3888000
